In [2]:
import requests
from lxml import etree 
import time 
import logging
import os
import sqlite3
import pathlib
import glob
import gzip 
import csv 

import pandas as pd
import xmltodict

URL_MERE_PLAN_DE_COMPTE = "http://odm-budgetaire.org/composants/normes/"
PLAN_COMPTE = "planDeCompte.xml"

def _recherche_plan_de_compte(annee, nomenclature) : 
 url_dossier = URL_MERE_PLAN_DE_COMPTE + f"{annee}/" + f"{nomenclature.split('-')[0]}/" + f"{nomenclature.split('-')[1]}/"
 return url_dossier

base_a = _recherche_plan_de_compte("2020", "M57-M57")
requete_base= requests.get(base_a + PLAN_COMPTE)
racine_base = etree.fromstring(requete_base.content)
enfants_base = racine_base.getchildren()
enfants_base

[<Element Nature at 0x1b13511fd00>, <Element Fonction at 0x1b1542561c0>]

enfant_nature_chap = 54

enfant_nat_compte = 3067

In [3]:
enfant_nature = enfants_base[0]

Nature_Chapitre = enfant_nature[0].xpath(".//*[@Code]")
Liste_Nature_Chapitre = []
for i in Nature_Chapitre : 
    i.attrib['origine'] = 'Nature_Chapitre'
    Liste_Nature_Chapitre.append(i.attrib)
Nature_Chapitre_df = pd.DataFrame(Liste_Nature_Chapitre)
Nature_Chapitre_df.loc[Nature_Chapitre_df['Code'] == '45411']

,Code,Lib_court,Libelle,Section,Special,TypeChapitre,origine
42,45411,Travaux effectués d'office,Travaux effectués d'office,I,0,C,Nature_Chapitre


In [4]:
Nature_Compte = enfant_nature[1].xpath(".//*[@Code]")
Liste_Nature_Compte = []
for i in Nature_Compte : 
    i.attrib['origine'] = 'Nature_Compte'
    Liste_Nature_Compte.append(i.attrib)
Nature_Compte_df = pd.DataFrame(Liste_Nature_Compte)
Nature_Compte_df.loc[Nature_Compte_df['Code'] == '45411']

,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis,origine
1780,45411,0,040,041,45411,Dépenses,Dépenses,0,040,041,45411,NaN,NaN,NaN,Nature_Compte


In [5]:
Fonction_chapitre = enfants_base[1].getchildren()[0].xpath(".//*[@Code]")
Fonction_compte = enfants_base[1].getchildren()[1].xpath(".//*[@Code]")
Fonction_ret = enfants_base[1].getchildren()[2].xpath(".//*[@Code]")

In [6]:
Liste_Fonction_Chapitre= []
for i in Fonction_chapitre:
    i.attrib['origine'] = 'Fonction_Chapitre'
    Liste_Fonction_Chapitre.append(i.attrib)
Fonction_chapitre_df = pd.DataFrame(Liste_Fonction_Chapitre)

Liste_Fonction_Compte = []
for i in Fonction_compte:
    i.attrib['origine'] = 'Fonction_Compte'
    Liste_Fonction_Compte.append(i.attrib)
Fonction_Compte_df = pd.DataFrame(Liste_Fonction_Compte)

Liste_Fonction_Ret = []
for i in Fonction_ret:
    i.attrib['origine'] = 'Fonction_Ret'
    Liste_Fonction_Ret.append(i.attrib)
Fonction_ret_df = pd.DataFrame(Liste_Fonction_Ret)


check duplicata dans CODE

In [7]:
df_transcode_M57 = pd.concat([Nature_Chapitre_df, Nature_Compte_df, Fonction_chapitre_df, Fonction_Compte_df, Fonction_ret_df], ignore_index=True)
df_transcode_M57.shape
df_transcode_M57[['Code','origine']].duplicated().any()


False

Création de Fonction de Transcodage 

In [18]:
enfants_base[0].getchildren()[1].xpath(".//*[@Code]")

[<Element Compte at 0x1b134ec12c0>,
 <Element Compte at 0x1b134ec37c0>,
 <Element Compte at 0x1b134ec08c0>,
 <Element Compte at 0x1b134ec0f80>,
 <Element Compte at 0x1b134ec1cc0>,
 <Element Compte at 0x1b134ec0a00>,
 <Element Compte at 0x1b134ec0b00>,
 <Element Compte at 0x1b134ec1840>,
 <Element Compte at 0x1b134ec2200>,
 <Element Compte at 0x1b134ec1e80>,
 <Element Compte at 0x1b134ec15c0>,
 <Element Compte at 0x1b134ec3b40>,
 <Element Compte at 0x1b134ea1dc0>,
 <Element Compte at 0x1b134ea3ec0>,
 <Element Compte at 0x1b134ea2780>,
 <Element Compte at 0x1b134ea1280>,
 <Element Compte at 0x1b134ea0580>,
 <Element Compte at 0x1b134ea3c00>,
 <Element Compte at 0x1b134ea06c0>,
 <Element Compte at 0x1b134ea1440>,
 <Element Compte at 0x1b134ea1300>,
 <Element Compte at 0x1b134ea3d40>,
 <Element Compte at 0x1b134ea05c0>,
 <Element Compte at 0x1b134ea0440>,
 <Element Compte at 0x1b134ea1d40>,
 <Element Compte at 0x1b134ea0740>,
 <Element Compte at 0x1b134ea0780>,
 <Element Compte at 0x1b134e